# Lab 4

In [1]:
import numpy as np
import pandas as pd 
import seaborn as sns 
import matplotlib as mp
import matplotlib.pyplot as plt 

from scipy import stats
from textwrap import wrap

pd.set_option('precision', 10)

%matplotlib inline
%config InlineBackend.figure_format = 'pdf'

In [2]:
CL = pd.read_csv('/Users/kev/Downloads/BCMB 301A Lab Techniques/Lab 4 - Partial Purification of beta-galactosidase from E. coli CSH66 Cell-Free Lysate/Data/Crude Lysate.csv',
                 header=1)

SN = pd.read_csv('/Users/kev/Downloads/BCMB 301A Lab Techniques/Lab 4 - Partial Purification of beta-galactosidase from E. coli CSH66 Cell-Free Lysate/Data/Subernatent.csv', 
                 header=1)

SE = pd.read_csv('/Users/kev/Downloads/BCMB 301A Lab Techniques/Lab 4 - Partial Purification of beta-galactosidase from E. coli CSH66 Cell-Free Lysate/Data/SEC.csv',
                 header=1)

DE = pd.read_csv('/Users/kev/Downloads/BCMB 301A Lab Techniques/Lab 4 - Partial Purification of beta-galactosidase from E. coli CSH66 Cell-Free Lysate/Data/DEAE.csv', 
                 header=1)

BF = pd.read_csv('/Users/kev/Downloads/BCMB 301A Lab Techniques/Lab 4 - Partial Purification of beta-galactosidase from E. coli CSH66 Cell-Free Lysate/Data/Bradford.csv', 
                 header=1)

MW = pd.read_csv('/Users/kev/Downloads/BCMB 301A Lab Techniques/Lab 4 - Partial Purification of beta-galactosidase from E. coli CSH66 Cell-Free Lysate/Data/MWvDist.csv', 
                 header=1)

In [3]:
CL_ave = pd.DataFrame(CL.iloc[:,1:].mean(axis=1))
CL_ave.columns = ['CL Ave']
CL = CL.join(CL_ave)

SN_ave = pd.DataFrame(SN.iloc[:,1:].mean(axis=1))
SN_ave.columns = ['CN Ave']
SN = SN.join(SN_ave)

## Absorbances

In [4]:
Ave = pd.concat([CL_ave, SN_ave, SE.iloc[:,1], DE.iloc[:,1]], sort=False, ignore_index=True, axis=1)
Ave.columns = ['Lysate', 'Supernatent', 'SEC', 'DEAE']
Ave

,Lysate,Supernatent,SEC,DEAE
0,0.0000,0.0000,0.000,0.000
1,0.0690,0.0105,0.372,0.388
2,0.1375,0.0210,0.557,0.626
3,0.2055,0.0325,0.636,0.742
4,0.2760,0.0425,0.660,0.794
5,0.3440,0.0535,0.671,0.811
6,0.4255,0.0645,0.673,0.819
7,0.4845,0.0755,0.675,0.821
8,0.5585,0.0880,0.677,0.825
9,0.6330,0.0995,0.679,0.824


In [5]:
Diff = Ave.diff()
Diff = Diff.loc[1:,:]
Diff

,Lysate,Supernatent,SEC,DEAE
1,0.0690,0.0105,0.372,0.388
2,0.0685,0.0105,0.185,0.238
3,0.0680,0.0115,0.079,0.116
4,0.0705,0.0100,0.024,0.052
5,0.0680,0.0110,0.011,0.017
6,0.0815,0.0110,0.002,0.008
7,0.0590,0.0110,0.002,0.002
8,0.0740,0.0125,0.002,0.004
9,0.0745,0.0115,0.002,-0.001
10,0.0685,0.0120,0.000,0.000


In [6]:
Diff_mean = [Diff.loc[:,'Lysate'].mean(axis=0), Diff.loc[:,'Supernatent'].mean(axis=0),
             Diff.loc[0:8,'SEC'].mean(axis=0), Diff.loc[0:7,'DEAE'].mean(axis=0)]
Diff_mean = pd.DataFrame(Diff_mean).T
Diff_mean.columns = ['Lysate', 'Supernatent', 'SEC', 'DEAE']
Diff_mean

,Lysate,Supernatent,SEC,DEAE
0,0.0692083333,0.011125,0.084625,0.1172857143


## Standard curve for BSA

In [7]:
BS = BF.iloc[0:7,:].astype('float64')
BS.columns = [r'BSA ($\mu$l)', 'Set 1', 'Set 2']
BS_ave = pd.DataFrame(BS.iloc[:,1:].mean(axis=1))
BS_m = BS[r'BSA ($\mu$l)'] * 0.1
BS = pd.concat([BS, BS_ave, BS_m], ignore_index=True, sort=False, axis = 1)
BS.columns = [r'BSA ($\mu$l)', 'Set 1', 'Set 2', 'Ave', r'BSA ($\mu$g)']
BS

,BSA ($\mu$l),Set 1,Set 2,Ave,BSA ($\mu$g)
0,0.0,-0.003,0.003,0.0000,0.0
1,40.0,0.060,0.063,0.0615,4.0
2,80.0,0.114,0.120,0.1170,8.0
3,120.0,0.167,0.171,0.1690,12.0
4,200.0,0.253,0.303,0.2780,20.0
5,250.0,0.392,0.359,0.3755,25.0
6,300.0,0.457,0.456,0.4565,30.0


In [8]:
def plot_abs(Dat, X, Y, Abs, Yerr, ploterror=False):
    slope, intercept, r_value, p_value, std_err = stats.linregress(Dat[X], Dat[Y])

    plt.figure()
    sns.regplot(x = X, y=Y, data=Dat, 
                scatter_kws={'s':8}, 
                line_kws={'linewidth':2, 'label':r"A=({0:.3g})M+({1:.3g}); R$^2$={2:.3g}".format(slope, intercept, r_value**2)})
    
    if ploterror == True: 
        plt.errorbar(x=Dat[X], y=Dat[Y], yerr=Yerr, scatterpoints=None, linewidth=0, elinewidth=1, capsize=2,
                    label='Uncertainty: {0:.4f}'.format(Yerr))
    
#     F_title = r'Changes in average absorbance at {0} nm ($\bar{{A}}_{1}$) with respect to the quantity of BSA, with a 95% confidence interval'.format(Abs, {Abs})
#     plt.title("\n".join(wrap(F_title, 80)))
    plt.ylabel(r'$\bar{{A}}_{0}$'.format({Abs}))
    plt.legend(loc=2)
    plt.grid()
    return slope, intercept, r_value**2

In [9]:
BSs, BSi, BSr = plot_abs(Dat=BS, X=r'BSA ($\mu$g)', Y='Ave', Abs=595, Yerr=0)

<Figure size 432x288 with 1 Axes>

In [10]:
BP = BF.iloc[8:,:]
BP_ave = pd.DataFrame(BP.iloc[:, 1:].mean(axis=1))
BP_m = (BP_ave - BSi)/BSs
BP = pd.concat([BP, BP_ave, BP_m], axis=1, ignore_index=False, sort=False)
BP.columns = ['Sample', 'Set 1', 'Set 2', 'Ave', r'BSA ($\mu$l)']
BP

,Sample,Set 1,Set 2,Ave,BSA ($\mu$l)
8,Crude,0.125,0.118,0.1215,8.3652113124
9,Supernatant,-0.006,-0.007,-0.0065,-0.1632411821


## Enzyme table

In [11]:
Vol = [0.98, 3.4 , 2.5 , 0.9]
Dil = [2000, 20, 400, 200]

Volu = pd.DataFrame(Vol).T
Volu.columns = list(Diff_mean.columns)
Conc = pd.DataFrame(Diff_mean * 6 * Dil / 3 )
EnzT = pd.DataFrame(Conc * Vol)
ProC = pd.DataFrame((Diff_mean - BSi)/BSs)
ProT = pd.DataFrame(ProC * Vol)
SpAc = pd.DataFrame(EnzT / ProT )
ProR = pd.DataFrame(ProT / float(ProT['Lysate']) * 100)
EnzR = pd.DataFrame(EnzT / float(EnzT['Lysate']) * 100)
FdPR = pd.DataFrame(SpAc / float(SpAc['Lysate']))

EnzTab = pd.concat([Volu, Conc, EnzT, ProC, ProT, SpAc, ProR, EnzR, FdPR], axis=0, sort=False, ignore_index=True)

In [12]:
EnzTab

,Lysate,Supernatent,SEC,DEAE
0,0.9800000000,3.4000000000,2.5000000000,0.9000000000
1,276.8333333333,0.4450000000,67.7000000000,46.9142857143
2,271.2966666667,1.5130000000,169.2500000000,42.2228571429
3,4.8810941638,1.0110867493,5.9082840801,8.0844196287
4,4.7834722805,3.4376949476,14.7707102002,7.2759776658
5,56.7154256906,0.4401204944,11.4584876222,5.8030493058
6,100.0000000000,71.8660994775,308.7863655125,152.1066129200
7,100.0000000000,0.5576920714,62.3855803610,15.5633527170
8,1.0000000000,0.0077601550,0.2020347636,0.1023187120


In [13]:
EnzTab.rename(index={0:'Volume (ml)', 
                     1:'Enzyme activity (units/ml)',
                     2:'Total enzyme (units)',
                     3:'[Protein] (mg/ml)', 
                     4:'Total protein (mg)',
                     5:'Specific activity (units/mg)',
                     6:'Total protein relative to lysate (%)',
                     7:'Enzyme yield relative to lysate (%)',
                     8:'Fold purification relative to lysate'}, inplace=True)

EnzTab

,Lysate,Supernatent,SEC,DEAE
Volume (ml),0.9800000000,3.4000000000,2.5000000000,0.9000000000
Enzyme activity (units/ml),276.8333333333,0.4450000000,67.7000000000,46.9142857143
Total enzyme (units),271.2966666667,1.5130000000,169.2500000000,42.2228571429
[Protein] (mg/ml),4.8810941638,1.0110867493,5.9082840801,8.0844196287
Total protein (mg),4.7834722805,3.4376949476,14.7707102002,7.2759776658
Specific activity (units/mg),56.7154256906,0.4401204944,11.4584876222,5.8030493058
Total protein relative to lysate (%),100.0000000000,71.8660994775,308.7863655125,152.1066129200
Enzyme yield relative to lysate (%),100.0000000000,0.5576920714,62.3855803610,15.5633527170
Fold purification relative to lysate,1.0000000000,0.0077601550,0.2020347636,0.1023187120


In [14]:
__logBase10of2 = 3.010299956639811952137388947244930267681898814621085413104274611e-1
def RoundToSigFigs_fp( x, sigfigs ):
    """
    Rounds the value(s) in x to the number of significant figures in sigfigs.
    Return value has the same type as x.

    Restrictions:
    sigfigs must be an integer type and store a positive value.
    x must be a real value.
    """
    if not ( type(sigfigs) is int or type(sigfigs) is long or
             isinstance(sigfigs, np.integer) ):
        raise TypeError( "RoundToSigFigs_fp: sigfigs must be an integer." )

    if sigfigs <= 0:
        raise ValueError( "RoundToSigFigs_fp: sigfigs must be positive." )

#     if not np.isreal( x.all() ):
#         raise TypeError( "RoundToSigFigs_fp: x must be real." )

    xsgn = np.sign(x)
    absx = xsgn * x
    mantissa, binaryExponent = np.frexp( absx )

    decimalExponent = __logBase10of2 * binaryExponent
    omag = np.floor(decimalExponent)

    mantissa *= 10.0**(decimalExponent - omag)

#     if mantissa < 1.0:
#         mantissa *= 10.0
#         omag -= 1.0

    return xsgn * np.around( mantissa, decimals=sigfigs - 1 ) * 10.0**omag

In [15]:
RoundToSigFigs_fp(EnzTab, 3)

,Lysate,Supernatent,SEC,DEAE
Volume (ml),0.98,3.40000,2.500,0.900
Enzyme activity (units/ml),277.00,0.44500,68.000,46.900
Total enzyme (units),271.00,1.51000,169.000,42.200
[Protein] (mg/ml),4.88,1.01000,5.910,8.100
Total protein (mg),4.78,3.44000,14.800,7.280
Specific activity (units/mg),56.70,0.44000,11.500,5.800
Total protein relative to lysate (%),100.00,72.00000,309.000,152.000
Enzyme yield relative to lysate (%),100.00,0.56000,62.400,15.600
Fold purification relative to lysate,1.00,0.00776,0.202,0.102


## Distance traveled vs molecular weight graph

### Image processing 

In [16]:
import scipy as sci
import skimage as ski
import PIL as pil

from PIL import Image
from scipy.ndimage import gaussian_filter
from skimage import data
from skimage import data, img_as_float
from skimage.morphology import reconstruction
from skimage.segmentation import chan_vese

In [17]:
SD = Image.open('/Users/kev/Downloads/BCMB 301A Lab Techniques/Lab 4 - Partial Purification of beta-galactosidase from E. coli CSH66 Cell-Free Lysate/Data/Scan2.png',)

In [18]:
fig, ax = plt.subplots(1, 1)
ax.imshow(np.array(SD), aspect='equal')
plt.show()
np.shape(np.array(SD))

<Figure size 432x288 with 1 Axes>

(2898, 4526, 4)

In [19]:
SD_gray = ski.color.rgb2gray(np.array(SD))

seed_SDg = np.copy(SD_gray)
seed_SDg[1:-1, 1:-1] = SD_gray.min()
mask_SDg = SD_gray

dilated_SDg = reconstruction(seed_SDg, mask_SDg, method='dilation')

In [20]:
fig, ax_sdg = plt.subplots(figsize=(20, 30))
plt.imshow((np.max(SD_gray)- SD_gray - dilated_SDg), aspect='equal', 
           cmap='gray_r', vmax=0.4,
          extent=(0, np.shape(SD_gray)[1]*25.4/1200, 
                  np.shape(SD_gray)[0]*25.4/1200, 0))

# Annotate names
MW_n = ['Molcular weight markers',
       r'$\beta$-Galactosidase standard',
       'Crude lysate (1/10 dilution)',
       'Suprnatant (1/01 dilution)',
       'SEC fraction preceeding SEC pool',
       'SEC pool',
       'SEC fraction following SEC pool',
       'DEAE flow through',
       'DEAE 0.2M KCl fraction',
       'DEAE 0.4M KCl Pool',
       '190 kDa', '135 kDa', '100 kDa', '75 kDa', '58 kDa', 
       '46 kDa', '35 kDa', '25 kDa', '22 kDa', '17 kDa' ]

MW_nxi = [6, 16, 24, 33, 41, 50, 59, 68, 77, 86,
         -13, -13, -13, -12, -12, -12, -12, -12, -12, -12]
MW_nyf = [-17, -17.5, -18.5, -17.75, -21.5, -8.75, -20.5, -14, -16.25, -14.5,
         5.5, 12, 15.5, 19.5, 22.5, 29.5, 36.5, 44.5, 48, 52.25]
MW_nr  = [30, 30, 30 , 30, 30 , 30, 30, 30, 30, 30,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

for i in range(0, len(MW_n)):
    plt.annotate(MW_n[i], xy=(MW_nxi[i], MW_nyf[i]), annotation_clip=False,
                fontsize=24, rotation=MW_nr[i], zorder=10)\

# Annotate MW
MW_xf  = [-4, -4, -4, -4, -4, -4, -4, -4, -4, -4]
MW_xf2 = [7, 17, 25, 34, 42, 51, 60, 69, 78, 87]
MW_xi  = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 
MW_xi2 = [7,17,25,34,42,51,60,69,78, 87]
MW_yf  = [9.5, 11.5, 15, 19, 22, 29, 36, 44, 47.5, 49.5]
MW_yf2 = [0, 0,  0,  0,  0,  0,  0,  0,  0, 0]
MW_yi  = [5, 11.5, 15, 19, 22, 29, 36, 44, 47.5, 52] 
MW_yi2 = [ -4,   -4, -4, -4, -4, -4, -4, -4, -4,-4]
MW_a   = [-90, 90, 90, 90, 90, 90, 90, 90, 90, 90]
MW_f   = [-0.4, -0.4, -0.4, -0.4, -0.4, -0.4, -0.4, -0.4, -0.4, -0.4]

for i in range(0, len(MW_yi2)):
    plt.annotate('', xy=(MW_xi2[i], MW_yf2[i]), xytext=(MW_xf2[i], MW_yi2[i]), fontsize=24,
                 annotation_clip=False,
                arrowprops=dict(arrowstyle='->'))

for i in range(0, len(MW_yi)):
    plt.annotate('', xy=(MW_xi[i], MW_yf[i]), xytext=(MW_xf[i], MW_yi[i]),
                fontsize=24, annotation_clip=False,
                arrowprops=dict(arrowstyle='->',facecolor='black',
                         connectionstyle="bar,angle={0},fraction={1}".format(MW_a[i], MW_f[i])))
    
plt.annotate('Protein migration direction', xy=(106, 18), fontsize=24,
                annotation_clip=False, rotation=90)

plt.annotate('', xy=(105, 60), xytext=(105, 1), fontsize=48,
             annotation_clip=False, 
             arrowprops=dict(facecolor='black', width=2))

ax_sdg.yaxis.tick_right()
ax_sdg.yaxis.set_label_position("right")
plt.xticks(np.arange(0, np.shape(SD_gray)[1]*25.4/1200, step=10), fontsize=24)
plt.yticks(fontsize=24)
plt.xlabel('x distance (mm)', fontsize=24)
plt.ylabel('y distance (mm)', fontsize=24)
plt.grid(linewidth=2, linestyle='--')
plt.show()

/Users/kev/anaconda3/lib/python3.7/site-packages/matplotlib/patches.py:3063: RuntimeWarning: invalid value encountered in double_scalars
  ddx, ddy = dx / dd2, dy / dd2


<Figure size 1440x2160 with 1 Axes>

### Generate graph

In [21]:
# Crop the image
MWM = np.array(SD)[20:2500, 350:500]
fig, ax2 = plt.subplots(1, 5)#  figsize=(8, 20))
for i in range(0, 4):
        ax2[i].imshow(np.max(MWM) - MWM[:,:,i] - MWM[:,:,3])
        
ax2[4].imshow(MWM)
plt.tight_layout()
plt.show()

<Figure size 432x288 with 5 Axes>

In [22]:
fig, ax3 = plt.subplots(3, 1)
MWM_gray = (ski.color.rgb2gray(MWM)).T
MWM_gray2 = np.max(MWM_gray) - MWM_gray
ax3[0].imshow(MWM_gray2, cmap='gray_r')
ax3[1].plot(MWM_gray2[100])
ax3[2].plot(np.nanmean(MWM_gray2, axis=0))
plt.tight_layout()
plt.show()

np.shape(MWM_gray2)

<Figure size 432x288 with 3 Axes>

(150, 2480)

In [23]:
image = MWM_gray2
seed = np.copy(image)
seed[:, 1:-1] = image.min()
mask = image

dilated = reconstruction(seed, mask, method='dilation')

fig, ax4 = plt.subplots(5, 1, figsize=(6, 8))

ax4[0].imshow(image, cmap='gray')
ax4[0].set_title('original image')
# ax4[0].axis('off')

ax4[1].imshow(dilated, vmin=image.min(), vmax=image.max(), cmap='gray')
ax4[1].set_title('dilated')
# ax4[1].axis('off')

ax4[2].imshow(image - dilated, cmap='gray')
ax4[2].set_title('image - dilated')
# ax4[2].axis('off')

ax4[3].plot((image - dilated)[100])
ax4[4].plot(np.nanmean(image - dilated, axis=0))

fig.tight_layout()

<Figure size 432x576 with 5 Axes>

In [24]:
image = MWM_gray2
Res = 1
Test = np.zeros((np.shape(MWM_gray2)[0], np.shape(MWM_gray2)[1]*Res + 1))

# Find the local maxima along each row
n = 0
for i in range(0, np.shape(MWM_gray2)[0]):
    x = (image - dilated)[i]
    peaks, _ = sci.signal.find_peaks(x, height=0.1, width=4, distance=50)
    # Filter out the bad rows (extra/insufficient data)
    if 2 < len(peaks) <= 11 and 0 < np.min(peaks) <= 100:
        Peaks = ((peaks - np.min(peaks)))# /np.max(peaks))*Res
#         Peaks = [int(i) for i in Peaks]
        for j in range(0, len(Peaks)):
            Test[i-n][int(Peaks[j])] += 1 
    else:
        n += 1

In [25]:
# plt.imshow(Test[:])
# plt.show()
# np.shape(Test[:]), np.shape(Test)

In [26]:
# Basically a histogram 
Test2 = np.sum(Test[0:-n], axis=0)

# plt.plot(Test2[1:])
norm = np.max(np.nanmean(image - dilated, axis=0))
# plt.plot(np.nanmean(image - dilated, axis=0)[80:] * np.max(Test2[1:]) / norm)
# plt.show()

In [27]:
Test3 = (Test2 > 1) * Test2
plt.figure(figsize=(16, 4))
plt.plot(Test3[1::Res])
plt.grid()
# plt.xticks(np.arange(0, len(Test3), step=100))
plt.show()

<Figure size 1152x288 with 1 Axes>

In [28]:
cuts = [300, 400, 510, 700, 900, 1100, 1500, 1800, 2100, 2200, 2300]
Cuts = [int(i * Res) for i in cuts]

# fig, axT = plt.subplots(len(Cuts)-1, 1, figsize=(4, 20))
AVSD = np.zeros((10, 2))
for i in range(0, len(Cuts)-1):
    Range = np.arange(Cuts[i], Cuts[i+1])
    Meat = Test3[Cuts[i]:Cuts[i+1]]
    if np.sum(Meat) > 0: 
        AVER = np.sum(Meat*Range)/np.sum(Meat)
        STDV = np.std(Meat*(Range-AVER))
    #     axT[i].plot(np.arange(Cuts[i], Cuts[i+1]),  Test3[Cuts[i]:Cuts[i+1]])
    #     axT[i].set_title('Ave: {0:.3g}, StDev: {1:.3g}'.format(AVER, STDV))
        AVSD[i] += np.array([AVER, STDV])
    else:
        AVSD + 0
    
# plt.tight_layout()
# plt.show()

In [29]:
PPI = 1200
MPI = 25.4
Ave_std = pd.DataFrame(AVSD / PPI * MPI)
Ave_std.columns = ['Average', 'StDev']
Mass = pd.DataFrame([190, 135, 100, 75, 58, 46, 32, 25, 22, 17])
Mass.columns = ['kDa']
Mass_log = pd.DataFrame(np.log10([190, 135, 100, 75, 58, 46, 32, 25, 22, 17]))
Mass_log.columns = ['log(kDa)']
Ave_std2 = pd.concat([Ave_std, Mass, Mass_log], axis=1, sort=False, ignore_index=False)
Ave_std2

,Average,StDev,kDa,log(kDa)
0,7.5548717949,0.1592601029,190,2.2787536010
1,9.7021315789,0.1312836006,135,2.1303337685
2,12.9042583333,0.1246152523,100,2.0000000000
3,17.4572083333,0.0837940053,75,1.8750612634
4,20.9150185185,0.0567513109,58,1.7634279936
5,27.3291904762,0.0896955897,46,1.6627578317
6,34.7967549020,0.0798860180,32,1.5051499783
7,42.0643921569,0.0461823407,25,1.3979400087
8,45.7297692308,0.0617843241,22,1.3424226808
9,47.5727058824,0.1406619167,17,1.2304489214


In [30]:
ADs, ADi, ADr = plot_abs(Dat=Ave_std2, X='log(kDa)', Y='Average', Abs=595, Yerr=0)
# plt.errorbar(x=Ave_std2['log(kDa)'], y=Ave_std2['Average'], yerr=Ave_std2['StDev'], 
#             scatterpoints=None, linewidth=0, elinewidth=2, capsize=3, label='Standard Deviation',
#             zorder=0)
plt.xlabel(r'Protein mass ($\log$(kDa))')
plt.ylabel('Displacement (mm)')
plt.legend(['D = ({0:.1f})M+({1:.3g}); R$^2$={2:.3g}'.format(ADs, ADi, ADr)],loc=1)
# plt.legend(loc=3)

<Figure size 432x288 with 1 Axes>

### Weight of $\beta$-Gal in lysate

In [31]:
BG = SD_gray[100:730,1000:1400]
def Weight_calc(A, H=30, W=4, D=50, slope=ADs, intercept=ADi):
    plt.imshow(A, cmap='gray')
    A_ave = (-np.sum(A, axis=1)+np.max(np.sum(A, axis=1))).T
    A_peaks, __ = sci.signal.find_peaks(A_ave, height=H, width=W, distance=D)
    plt.figure()
    plt.plot(A_ave)
    plt.scatter(A_peaks, A_ave[A_peaks], color='orange', s=5, zorder=5)
    plt.show()
    A_weight = 10**((((A_peaks - A_peaks.min()) * 25.4 / 1200) - intercept)/slope)
    return A_weight

In [32]:
Weight_calc(BG)

<Figure size 432x288 with 1 Axes>

<Figure size 432x288 with 1 Axes>

array([222.87358391, 188.33891309, 136.84066127, 114.97212747])

### $\beta$-Gal in standard 

In [33]:
Weight_calc(SD_gray[100:500,650:980])

<Figure size 432x288 with 1 Axes>

<Figure size 432x288 with 1 Axes>

array([222.87358391, 179.43450464])